# FINAL PROJECT

**Names:** -- Diana Zepeda Tatengo and Victor Hugo Gomez Soto --

**e-mails:** -- diana.zepeda6085@alumnos.udg.mx and victor.gomez2701@alumnos.udg.mx --

# MODULES

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
from datetime import datetime

# WIDGETS

In [ ]:
# Carga de archivo
upload = widgets.FileUpload(accept='.csv,.xlsx', multiple=False)

# Selectors
column_selector = widgets.SelectMultiple(options=[], description="Columns:", layout=widgets.Layout(width='100%'))
compare_with_selector = widgets.Dropdown(options=[], description="Compare with:", layout=widgets.Layout(width='100%'))
date_range = widgets.SelectionRangeSlider(
    description='Range:',
    options=[datetime(2000, 1, 1), datetime(2000, 12, 31)],
    value=(datetime(2000, 1, 1), datetime(2000, 12, 31)),
    layout=widgets.Layout(width='100%'),
    continuous_update=False
)

# Función de carga
def on_file_upload(change):
    global df_global
    output_info.clear_output()
    output_table.clear_output()
    output_plot.clear_output()

    if upload.value:
        uploaded_file = upload.value[0]
        content = uploaded_file['content']
        filename = uploaded_file['name']


        try:
            if filename.endswith('.csv'):
                df = pd.read_csv(pd.io.common.BytesIO(content))
            else:
                df = pd.read_excel(pd.io.common.BytesIO(content))

            df.columns = df.columns.str.strip()

            if all(col in df.columns for col in ["Year", "Month", "Day"]):
                df["Fecha"] = pd.to_datetime(df[["Year", "Month", "Day"]])
                df = df.sort_values("Fecha")
            else:
                raise ValueError("Se requieren las columnas 'Year', 'Month' y 'Day'.")

            df_global = df

            columnas_numericas = df.select_dtypes(include='number').columns.tolist()
            columnas_numericas = [col for col in columnas_numericas if col not in ["Year", "Month", "Day"]]
            column_selector.options = columnas_numericas
            comparar_con_selector.options = columnas_numericas
            if columnas_numericas:
                column_selector.value = tuple(columnas_numericas[:1])
                comparar_con_selector.value = columnas_numericas[1] if len(columnas_numericas) > 1 else None

            fechas_unicas = df["Fecha"].sort_values().unique()
            date_range.options = list(fechas_unicas)
            date_range.value = (fechas_unicas[0], fechas_unicas[-1])

            with output_info:
                print(f"✅ Archivo cargado: {filename}")
        except Exception as e:
            with output_info:
                print(f"❌ Error: {e}")
# Eventos
upload.observe(on_file_upload, names='value')

# Panel-style Layout: Columns
sidebar = widgets.VBox([
    widgets.Label("📁 Upload File"),  
    upload  
    widgets.Label("📌 Columns to display"),
    column_selector,
    widgets.Label("📈 Compare With"),
    compare_with_selector,
    widgets.Label("📅 Date Range"),
    date_range
], layout=widgets.Layout(width='30%', padding='10px'))

# DASHBOARD

In [ ]:
# Header
display(HTML("""
<div style="background-color:#1976D2; padding: 10px 20px; color: white; font-size: 24px; font-family: Arial; text-align: center;">
    🌍📊🌦️<b>Climate Data Dashboard</b>🌡️🌞🌧️
</div>
"""))

# Display widgets
display(sidebar)